In [2]:
# Listing Datasets and cleaning the peep registry file
import pandas as pd

df = pd.read_csv("pp_peep_registry.csv")

peep_stats = df["peep_status"].isna()
df["peep_status"] = df["peep_status"].fillna(value="Not Available")
df["peep_required"] = df["peep_required"].astype(str).str.lower()

yes_values = ["y", "yes", "true"]
no_values = ["n", "no", "false"]

df["peep_required"] = df["peep_required"].apply(
    lambda x: "Yes" if x in yes_values else ("No" if x in no_values else "Not Specified")
)

mobility_category = df["mobility_category"].isna()
df["mobility_category"] = df["mobility_category"].fillna(value="Category Not Specified")

df["last_review_date"] = pd.to_datetime(df["last_review_date"], format="mixed", dayfirst=True, errors='coerce')

property_names = {"London HQ": "LDN-HQ", "Bristol Harbourside": "BRI-01", "Edinburgh Exchange" : "EDI-EX", "Manchester City Campus" : "MCH-CC", "Leeds Arena Point": "LDS-AR", "Birmingham Paradise": "BHM-PR", "Cardiff Bay": "CDF-BY", "Belfast Lagan": "BEL-LG", "Norwich Quarter": "NRW-QP", "Glasgow Clyde" : "GLW-CL"}
df["property_code"] = df["property_code"].replace(property_names)
df["property_code"] = df["property_code"].str.upper()
df = df[~((df["peep_required"] == "No") & (df["peep_status"] == "Not Available"))]
bad_statuses = ["Not Available", "Pending", "Expired", "Revoked"]
df["risk_status"] = df.apply(
    lambda row: "Potential" if (row["peep_status"] in bad_statuses and row["peep_required"] == "Yes") else "Not Potential",
    axis=1
)
df = df.drop_duplicates()
df.head(100)
# df.to_csv("cleaned_peep_registry.csv", index=False)

,person_id,full_name,email,property_code,peep_required,peep_status,mobility_category,last_review_date,risk_status
0,P100001,Taylor Smith,taylor.smith@mailinator.com,LDS-AR,Yes,Approved,Wheelchair,2025-09-03,Not Potential
1,P100002,Sam Smith,sam.smith@example.com,MCH-CC,No,Pending,Wheelchair,2025-09-14,Not Potential
2,P100003,Cameron Rodriguez,cameron.rodriguez@example.com,EDI-EX,No,Approved,Category Not Specified,2025-06-04,Not Potential
3,P100004,Taylor Anderson,taylor.anderson@testmail.co,GLW-CL,Yes,Approved,Ambulatory,2025-07-03,Not Potential
4,P100005,Parker Anderson,parker.anderson@mailinator.com,MCH-CC,Yes,Approved,Ambulatory,2025-02-28,Not Potential
...,...,...,...,...,...,...,...,...,...
98,P100099,Jordan Williams,jordan.williams@testmail.co,BHM-PR,No,Approved,Ambulatory,2025-06-17,Not Potential
99,P100100,Avery Miller,avery.miller@testmail.co,BEL-LG,No,Expired,VisionImpaired,2025-08-03,Not Potential
100,P100101,Riley Taylor,riley.taylor@mailinator.com,BHM-PR,No,Approved,Ambulatory,2025-06-22,Not Potential
101,P100102,Avery Thompson,avery.thompson@testmail.co,BHM-PR,No,Approved,Neurodivergent,2025-09-11,Not Potential


In [3]:
df2 = pd.read_csv("pp_property_occupancy_logs.csv")
df2["timestamp"] = pd.to_datetime(df2["timestamp"], format="mixed", errors="coerce", dayfirst=True)
# df2["timestamp"] = pd.to_datetime(df2["timestamp"], errors="coerce", utc=True)
# df2["timestamp"] = df2["timestamp"]

df2["property_code"] = df2["property_code"].replace(property_names)
df2["property_code"] = df2["property_code"].str.upper()
df2["property_code"] = df2["property_code"].replace("Unknown", "")
df2["property_code"] = df2["property_code"].fillna("Not Available")
df2["property_code"] = df2["person_id"].map(df.set_index("person_id")["property_code"])

df2.loc[df2["property_code"].isna()]

df2["property_code"] = df2["property_code"].fillna(df["person_id"].map(
    df.set_index("person_id")["property_code"]
))

df2 = df2.sort_values(["event_id"])
direction_in = df2[df2["direction"] == "IN"]
direction_out = df2[df2["direction"] == "OUT"]

in_count = direction_in.groupby("person_id").size()
out_count = direction_out.groupby("person_id").size()

visits = pd.DataFrame({
    "in_count" : in_count,
    "out_count": out_count
}).fillna(0)

visits["total_visits"] = visits[["in_count", "out_count"]].min(axis=1)

df2 = df2.merge(visits["total_visits"], on="person_id", how="left")

df2["property_code"] = df2["property_code"].fillna("Not Specified")
df_merged = df2.merge(df, on="person_id", how="inner")
df_merged = df_merged.drop(columns=["property_code_y"])
df_merged = df_merged.rename(columns={"property_code_x": "property_code", "total_visits": "peep_visits"})
peep_visits = df_merged.groupby("person_id").size().reset_index(name="total_vists_correct")
df_merged = df_merged.merge(peep_visits, on="person_id", how="inner")
df_merged.to_csv("cleaned_pp_property_occupancy_logs.csv", index=False)
df_merged.head(10)


C:\Users\Alhaan Ahmed\AppData\Local\Temp\ipykernel_4020\3699431209.py:2: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df2["timestamp"] = pd.to_datetime(df2["timestamp"], format="mixed", errors="coerce", dayfirst=True)


,event_id,timestamp,property_code,person_id,direction,access_point,peep_visits,full_name,email,peep_required,peep_status,mobility_category,last_review_date,risk_status,total_vists_correct
0,E0000001,2025-06-01 09:42:00,NRW-QP,P100638,IN,Turnstile 1,12.0,Casey Brown,casey.brown@example.com,Yes,Revoked,Ambulatory,2025-05-22,Potential,25
1,E0000002,2025-06-01 20:37:00+00:00,NRW-QP,P100638,OUT,Reception,12.0,Casey Brown,casey.brown@example.com,Yes,Revoked,Ambulatory,2025-05-22,Potential,25
2,E0000003,2025-06-01 10:20:00,BHM-PR,P100221,IN,Reception,7.0,Rowan Hernandez,rowan.hernandez@example.com,No,Approved,Neurodivergent,2025-09-21,Not Potential,15
3,E0000004,2025-06-01 14:31:00,BHM-PR,P100221,OUT,Turnstile 1,7.0,Rowan Hernandez,rowan.hernandez@example.com,No,Approved,Neurodivergent,2025-09-21,Not Potential,15
4,E0000005,2025-06-01 10:08:00,NRW-QP,P100429,IN,Reception,19.0,Alex Davis,alex.davis@mailinator.com,Yes,Pending,Ambulatory,2025-04-20,Potential,38
5,E0000006,2025-06-01 14:44:00,NRW-QP,P100429,OUT,Reception,19.0,Alex Davis,alex.davis@mailinator.com,Yes,Pending,Ambulatory,2025-04-20,Potential,38
6,E0000007,2025-06-01 11:03:00,EDI-EX,P100327,IN,Reception,11.0,Harper Davis,harper.davis@mailinator.com,No,Revoked,Neurodivergent,2025-02-03,Not Potential,24
7,E0000008,2025-06-01 18:42:00,EDI-EX,P100327,OUT,Turnstile 1,11.0,Harper Davis,harper.davis@mailinator.com,No,Revoked,Neurodivergent,2025-02-03,Not Potential,24
8,E0000009,2025-06-01 07:08:00,NRW-QP,P100073,IN,Turnstile 1,4.0,Morgan Garcia,morgan.garcia@example.com,Yes,Approved,Ambulatory,2025-05-02,Not Potential,9
9,E0000010,2025-06-01 15:09:00,NRW-QP,P100073,OUT,Reception,4.0,Morgan Garcia,morgan.garcia@example.com,Yes,Approved,Ambulatory,2025-05-02,Not Potential,9
